# 词向量
- 学习词向量的概念
- 使用 skip-thought 模型训练词向量
- 使用 pytorch dataset 和 dataloader
- 学习定义 pytorch 模型
- 学习torch。nn 中的 Module
  -  Embedding
- 学习常见的 pytorch 的 operations
  - bmm
  - logsigmoid
- 保存和读取 pytorch 模型

第二课使用的训练数据可以从以下链接下载到。

链接:https://pan.baidu.com/s/1tFeK3mXuVXEy3EMarfeWvg 密码:v2z5

在这一份notebook中，我们会（尽可能）尝试复现论文Distributed Representations of Words and Phrases and their Compositionality中训练词向量的方法. 我们会实现Skip-gram模型，并且使用论文中noice contrastive sampling的目标函数。

这篇论文有很多模型实现的细节，这些细节对于词向量的好坏至关重要。我们虽然无法完全复现论文中的实验结果，主要是由于计算资源等各种细节原因，但是我们还是可以大致展示如何训练词向量。

以下是一些我们没有实现的细节

subsampling：参考论文section 2.3

**两个模块的区别：**[torch.nn 和 torch.functional 的区别](https://blog.csdn.net/hawkcici160/article/details/80140059)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as tud

from collections import Counter
import numpy as np
import random
import math
import pandas as pd
import scipy
import sklearn
from sklearn.metrics.pairwise import cosine_similarity # 余弦相似度

USE_CUDA = torch.cuda.is_available()

#为了方式 随机数对数据结果产生的影响，这里设置所有的 random.seed 是 1

np.random.seed(1)
random.seed(1)
torch.manual_seed(1)

if USE_CUDA :
    torch.cuda.manual_seed(1)
    

# set some hyper parameters
C = 3 # context window
K = 100 # number of negative samples
NUM_EPOCHS = 3
BATCH_SIZE = 128
LEARNING_RATE = 0.2
EMBEDDING_DECENT = 150
VOCAB_SIZE = 30000 # the size of vocab ,not all word in the vocab

LOG_FILE = "word-embedding.log"

def word_tokenize(text):
    ''' split the sequence as words '''
    return text.split()


- 从文本文件中读取所有的文字，通过这些文本创建一个vocabulary
- 由于单词数量可能太大，我们只选取最常见的MAX_VOCAB_SIZE个单词
- 我们添加一个UNK单词表示所有不常见的单词
- 我们需要记录单词到index的mapping，以及index到单词的mapping，单词的count，单词的(normalized) frequency，以及单词总数。

In [8]:

with open('text8.train.txt','r')as file:
    text = file.read()
    #text = text.split()
text = [w for w in word_tokenize(text)]

''' 使用最常见的单词制作词典实际上是不合适的，因为文本未经过处理，一些意义不大的常用词可能需要剔除 ，这一步需要事先对 text 进行处理 '''
vocab = dict (Counter(text).most_common(VOCAB_SIZE-1)) # 这里 -1 是为了给不常用词留一个位置 
vocab['<UNK>'] = len(text) - np.sum(list(vocab.values())) # vocab.values() 获取所有词的出现的次数 

id2word = [w for w in vocab.keys()]
word2id = {w: i for i,w in enumerate(id2word)}


In [9]:
word_counts =  np.array([count for count in vocab.values()],dtype = np.float32)
word_freq = word_counts / np.sum(word_counts)
# 对词频进行 3/4 次方处理
word_freq = word_freq ** (0.75)
word_freq = word_freq / np.sum(word_freq)
print(word_freq.shape)

(30000,)


### 实现Dataloader

一个dataloader需要以下内容：

- 把所有text编码成数字，然后用subsampling预处理这些文字。
- 保存vocabulary，单词count，normalized word frequency
- 每个iteration sample一个中心词
- 根据当前的中心词返回context单词
- 根据中心词sample一些negative单词
- 返回单词的counts

这里有一个好的tutorial介绍如何使用[PyTorch dataloader](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html).
为了使用dataloader，我们需要定义以下两个function:

- ```__len__``` function需要返回整个数据集中有多少个item
- ```__get__``` 根据给定的index返回一个item

有了dataloader之后，我们可以轻松随机打乱整个数据集，拿到一个batch的数据等等。

In [10]:
class WordEmbeddingDataset(tud.Dataset):
    def __init__(self, text, word2id, id2word, word_freq, word_counts):
        ''' 初始化 '''
        ''' text: a list of words, all text from the training dataset
            word_to_idx: the dictionary from word to idx
            idx_to_word: idx to word mapping
            word_freq: the frequency of each word
            word_counts: the word counts
        '''
        super(WordEmbeddingDataset, self).__init__()
        
        # 语料 数字化  后面的 word2id['UNK'] 就是 当单词不在常用词列表里时 用 UNK 的 id 代替
        self.text_encoded = [word2id.get(word, word2id['<UNK>']) for word in text] 
        self.text_encoded = torch.Tensor(self.text_encoded).long() # 将 text_encoded 转换成 long tensor 类型
        
        self.word2id = word2id
        self.id2word = id2word
        self.word_freq = torch.Tensor(word_freq)
        self.word_counts = torch.Tensor(word_counts)
        
    def __len__(self,):
        ''' 数据集中 item 的数量 '''
        return len(self.text_encoded)
    def __getitem__(self,idx):
        ''' 根据 index 返回对应的 item'''
        center_word = self.text_encoded[idx] # 中心词
        
        # 获取周围词
        pos_indices = list(range(idx - C))+list(range(idx+1,idx+C+1)) #获取周围词列表 range 不会取到小于 0 的位置
        pos_indices = [i % len(self.text_encoded) for i in pos_indices] # 对于 idx + word_window +1 大于 text 的长度时特殊处理
        pos_words = self.text_encoded[pos_indices]
        '''
        
        # 获取周围词
        start = max(0,idx-C)
        end = min(len(self.text_encoded),(idx+1+C))
        pos_wrods = self.text_encoded[start:idx]+self.text_encoded[idx+1,end]
        '''
        
        ''' multinomial 函数的参数： 元素分布的概率， 需要采样的个数， 是否可以重复采样同一个单词
            self.word_freq：单词处理后的概率， 
            K * pos_words.shape[0]：每个正例单词需要采样 k 个负例单词
            replacement = True ： 是否可以重复采样同一个单词
        '''
        neg_words = torch.multinomial(self.word_freq, K * pos_words.shape[0], True)
        
        return center_word, pos_words, neg_words

#### 创建 dataset 和 dataloader

In [11]:
dataset = WordEmbeddingDataset( text, word2id, id2word, word_freq, word_counts)
dataloader = tud.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)      # shuffle :打乱顺序。 num_workers ：线程数量，多线程


torch.utils.data.DataLoader理解：https://blog.csdn.net/qq_36653505/article/details/83351808

### 定义 PyTorch 模型

In [7]:
class EmbeddingModel(nn.Module):
    def __init__(self, vocab_size, embed_size):
        super(EmbeddingModel, self).__init__()
        ''' 初始化输出和输出embedding
        '''
        self.vocab_size = vocab_size  #30000
        self.embed_size = embed_size  #100
        
        initrange = 0.5 / self.embed_size
        self.out_embed = nn.Embedding(self.vocab_size, self.embed_size, sparse=False)
        #模型输出nn.Embedding(30000, 100)
        self.out_embed.weight.data.uniform_(-initrange, initrange)
        #权重初始化的一种方法
        
        
        self.in_embed = nn.Embedding(self.vocab_size, self.embed_size, sparse=False)
         #模型输入nn.Embedding(30000, 100)
        self.in_embed.weight.data.uniform_(-initrange, initrange)
        #权重初始化的一种方法
        
        
    def forward(self, input_labels, pos_labels, neg_labels):
        '''
        input_labels:[batch_size]
        pos_labels:[batch_size,(window_size *2)]
        neg_labels:[batch_size,(window_size *2 *k)]
        '''
        input_embedding = self.in_embed(input_labels) #[batch_size, embed_size]
        input_embedding = input_embedding.unsqueeze(2) # 创建一个 1 的维度 [batch_size, embed_size，1]
        
        pos_embedding = self.in_embed(pos_labels) #[batch_size, window_size *2, embed_size]
        neg_embedding = self.in_embed(neg_labels) #[batch_size, window_size *2 *k, embed_size]
        
        # bmm :batch matrix multiply 维度 [b,c,d] * [b,d,e] = [b,c,e]
        pos_dot = torch.bmm(pos_embedding, input_embedding).sequeeze(2)#[batch_size, window_size *2, 1].squeeze ->[batch_size, window_size *2]
        neg_dot = torch.bmm(neg_embedding, -input_embedding).sequeeze(2) #[batch_size, window_size *2 *k]
        
        log_pos = F.logsigmoid(pos_dot).sum(1)
        log_neg = F.logsigmoid(neg_dot).sum(1)
        
        loss = log_pos + log_neg
        
        return -loss
        
    def input_embedding(self):
        ''' 获取 input embedding'''
        return self.in_embed.wight.data.cpu().numpy()

定义一个模型以及把模型移动到GPU

In [8]:
model = EmbeddingModel(VOCAB_SIZE, EMBEDDING_DECENT)
#得到model，有参数，有loss，可以优化了

if USE_CUDA:
    model = model.cuda()

下面是评估模型的代码，以及训练模型的代码

In [ ]:
def evaluate(filename, embedding_weights): 
    if filename.endswith(".csv"):
        data = pd.read_csv(filename, sep=",")
    else:
        data = pd.read_csv(filename, sep="\t")
    human_similarity = []
    model_similarity = []
    for i in data.iloc[:, 0:2].index:
        word1, word2 = data.iloc[i, 0], data.iloc[i, 1]
        if word1 not in word_to_idx or word2 not in word_to_idx:
            continue
        else:
            word1_idx, word2_idx = word_to_idx[word1], word_to_idx[word2]
            word1_embed, word2_embed = embedding_weights[[word1_idx]], embedding_weights[[word2_idx]]
            model_similarity.append(float(sklearn.metrics.pairwise.cosine_similarity(word1_embed, word2_embed)))
            human_similarity.append(float(data.iloc[i, 2]))

    return scipy.stats.spearmanr(human_similarity, model_similarity)# , model_similarity

def find_nearest(word):
    index = word_to_idx[word]
    embedding = embedding_weights[index]
    cos_dis = np.array([scipy.spatial.distance.cosine(e, embedding) for e in embedding_weights])
    return [idx_to_word[i] for i in cos_dis.argsort()[:10]]

训练模型：
- 模型一般需要训练若干个epoch
- 每个epoch我们都把所有的数据分成若干个batch
- 把每个batch的输入和输出都包装成cuda tensor
- forward pass，通过输入的句子预测每个单词的下一个单词
- 用模型的预测和正确的下一个单词计算cross entropy loss
- 清空模型当前gradient
- backward pass
- 更新模型参数
- 每隔一定的iteration输出模型在当前iteration的loss，以及在验证数据集上做模型的评估

In [ ]:
for i, (input_labels, pos_labels, neg_labels) in enumerate(dataloader):
        print(input_labels, pos_labels, neg_labels)
        if i>5:
            break

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)

for epoch in range(NUM_EPOCHS):
    for i, (input_labels, pos_labels, neg_labels) in enumerate(dataloader):
        input_labels = input_labels.long()
        pos_labels = pos_labels.long()
        neg_labels = neg_labels.long()
        if USE_CUDA:
            input_labels = input_labels.cuda()
            pos_labels = pos_labels.cuda()
            neg_labels = neg_labels.cuda()
        optimizer.zero_grad()
        loss = model(input_labels, pos_labels, neg_labels).mean() #求平均
        loss.backward()
        optimizer.step()
        
        
        #打印结果。
        if i % 100 == 0:
            with open(LOG_FILE, "a") as fout:
                fout.write("epoch: {}, iter: {}, loss: {}\n".format(e, i, loss.item()))
                print("epoch: {}, iter: {}, loss: {}".format(e, i, loss.item()))
            
        
        if i % 2000 == 0:
            embedding_weights = model.input_embeddings()
            sim_simlex = evaluate("simlex-999.txt", embedding_weights)
            sim_men = evaluate("men.txt", embedding_weights)
            sim_353 = evaluate("wordsim353.csv", embedding_weights)
            with open(LOG_FILE, "a") as fout:
                print("epoch: {}, iteration: {}, simlex-999: {}, men: {}, sim353: {}, nearest to monster: {}\n".format(
                    e, i, sim_simlex, sim_men, sim_353, find_nearest("monster")))
                fout.write("epoch: {}, iteration: {}, simlex-999: {}, men: {}, sim353: {}, nearest to monster: {}\n".format(
                    e, i, sim_simlex, sim_men, sim_353, find_nearest("monster")))
                
    embedding_weights = model.input_embeddings()
    np.save("embedding-{}".format(EMBEDDING_SIZE), embedding_weights)
    torch.save(model.state_dict(), "embedding-{}.th".format(EMBEDDING_SIZE))

In [ ]:
model.load_state_dict(torch.load("embedding-{}.th".format(EMBEDDING_SIZE)))

## 在 MEN 和 Simplex-999 数据集上做评估

In [ ]:
embedding_weights = model.input_embeddings()
print("simlex-999", evaluate("simlex-999.txt", embedding_weights))
print("men", evaluate("men.txt", embedding_weights))
print("wordsim353", evaluate("wordsim353.csv", embedding_weights))

## 寻找nearest neighbors

In [ ]:
for word in ["good", "fresh", "monster", "green", "like", "america", "chicago", "work", "computer", "language"]:
    print(word, find_nearest(word))

## 单词之间的关系

In [ ]:
man_idx = word_to_idx["man"] 
king_idx = word_to_idx["king"] 
woman_idx = word_to_idx["woman"]
embedding = embedding_weights[woman_idx] - embedding_weights[man_idx] + embedding_weights[king_idx]
cos_dis = np.array([scipy.spatial.distance.cosine(e, embedding) for e in embedding_weights])
for i in cos_dis.argsort()[:20]:
    print(idx_to_word[i])

In [56]:
a=[0,1,2,3,4,5,6,7,8,9]
b = a[:3]+a[4:6]


[0, 1, 2, 4, 5]
